## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-20-52-14 +0000,nyt,"June Wilkinson, Pinup Star and Screen Siren, I...",https://www.nytimes.com/2025/09/11/movies/june...
1,2025-09-11-20-44-26 +0000,wapo,"Bolsonaro found guilty in attempted coup, assa...",https://www.washingtonpost.com/world/2025/09/1...
2,2025-09-11-20-43-07 +0000,nyt,Carney Unveils Major Projects for Canada to Of...,https://www.nytimes.com/2025/09/11/world/canad...
3,2025-09-11-20-43-02 +0000,bbc,Bolsonaro found guilty - what you need to know,https://www.bbc.com/news/articles/cy4d409v2neo...
4,2025-09-11-20-42-16 +0000,nyt,"For Trump, Charlie Kirk Is a Deeply Personal Loss",https://www.nytimes.com/2025/09/11/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,kirk,69
25,charlie,63
21,trump,36
110,shooting,18
105,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
57,2025-09-11-19-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,229
300,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,221
244,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...,211
204,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free sp...,https://www.latimes.com/california/story/2025-...,198
324,2025-09-10-22-52-28 +0000,nypost,"Ghoulish Dems, liberals are blaming Trump, Cha...",https://nypost.com/2025/09/10/us-news/ghoulish...,194


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
57,229,2025-09-11-19-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
308,74,2025-09-10-23-40-09 +0000,nypost,UK ambassador Peter Mandelson told Jeffrey Eps...,https://nypost.com/2025/09/10/us-news/uk-ambas...
185,60,2025-09-11-13-50-47 +0000,latimes,FBI releases photos of 'person of interest' in...,https://www.latimes.com/world-nation/story/202...
244,55,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...
65,53,2025-09-11-18-52-58 +0000,nypost,Iryna Zarutska’s boyfriend shares his heartbre...,https://nypost.com/2025/09/11/us-news/iryna-za...
36,42,2025-09-11-19-47-56 +0000,nypost,"Desmond Holly, 16, ID’d as Evergreen High Scho...",https://nypost.com/2025/09/11/us-news/desmond-...
226,41,2025-09-11-10-00-00 +0000,latimes,California legislators strike last-minute deal...,https://www.latimes.com/california/story/2025-...
17,40,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
63,37,2025-09-11-18-54-10 +0000,nypost,Paramount Skydance prepares Ellison-backed bid...,https://nypost.com/2025/09/11/business/paramou...
295,37,2025-09-11-00-22-00 +0000,wsj,Senate Republicans voted down a measure to rel...,https://www.wsj.com/politics/policy/senate-rep...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
